In [1]:
import h5py 
import dxchange
import sys

sys.path.insert(0, '..')
from utils import *
from rec import Rec

detector_pixelsize = 3.03751e-6
focusToDetectorDistance = 1.28  # [m]
# adjustments for the cone beam
sx0 = 1.286*1e-3
z10 = 5.5*1e-3#/entry_0000: AtomiumS2 - AtomiumS2_codedht_random_007nm/TOMO/sx0 1.286 mm
z1 = z10-sx0
z2 = focusToDetectorDistance - z1
distance = (z1 * z2) / focusToDetectorDistance
magnification = focusToDetectorDistance / z1
voxelsize = float(cp.abs(detector_pixelsize / magnification))
print(voxelsize)

astropy module not found
olefile module not found


1.0000052453125e-08


In [2]:

# path = '/data/vnikitin/ESRF/ID16A/20240924/Chip/Chip_codedht_random2_010nm/'
# path_out = '/data/vnikitin/ESRF/ID16A/20240924_h5/Chip/Chip_codedht_random2_010nm/'

# def find_angle(filename):    
#     with open(filename, 'r',encoding="latin-1") as file:
#         for line in file:
#             if "motor_pos" in line:
#                 print(line.split())
#                 return float(line.split()[2])    
# with  h5py.File(f'{path_out}/data.h5','a') as fid:
    
#     for id in range(7200):        
#         fname = f'{path}Chip_codedht_random2_010nm{id:04}.edf'
        
#         ang = find_angle(fname)        
#         fid['exchange/theta'][id]=ang    
#         print(id,ang)
#         # pshifts = find_pos_shifts(fname)
#         # print(fname)
#         # print(ang)  
#         # print(pshifts)  
#         # data[id] = dxchange.read_edf(fname)[0]
#         # theta[id] = ang
#         # pos_shifts[id] = pshifts

['motor_pos', '=', '0', '0.56577', '30', '-2.3428', '0.7093', '-208.025', '2.51', ';']
0 0.0
['motor_pos', '=', '0.0250169', '0.56577', '30', '-2.3161', '0.6665', '-207.977', '2.56', ';']
1 0.0250169
['motor_pos', '=', '0.0500339', '0.56577', '30', '-2.4244', '0.6417', '-207.952', '2.5425', ';']
2 0.0500339
['motor_pos', '=', '0.0750508', '0.56577', '30', '-2.3096', '0.7631', '-207.982', '2.5375', ';']
3 0.0750508
['motor_pos', '=', '0.100068', '0.56577', '30', '-2.4378', '0.6585', '-208.002', '2.5125', ';']
4 0.100068
['motor_pos', '=', '0.124932', '0.56577', '30', '-2.4651', '0.5981', '-208.023', '2.5075', ';']
5 0.124932
['motor_pos', '=', '0.149949', '0.56577', '30', '-2.2581', '0.691', '-208.045', '2.535', ';']
6 0.149949
['motor_pos', '=', '0.174966', '0.56577', '30', '-2.4293', '0.6666', '-208.02', '2.5575', ';']
7 0.174966
['motor_pos', '=', '0.199983', '0.56577', '30', '-2.4154', '0.7419', '-208.005', '2.51', ';']
8 0.199983
['motor_pos', '=', '0.225', '0.56577', '30', '-2.368

In [4]:

def find_angle(filename):    
    with open(filename, 'r',encoding="latin-1") as file:
        for line in file:
            if "motor_pos" in line:
                print(line.split())
                return float(line.split()[2])        

def find_pos_shifts(filename):    
    with open(filename, 'r',encoding="latin-1") as file:
        for line in file:
            if "motor_pos" in line:
                print(line.split())
                return np.array([float(line.split()[5]),float(line.split()[6])])                

path = '/data/vnikitin/ESRF/ID16A/20240924/Chip/Chip_codedht_random2_010nm/'
path_out = '/data/vnikitin/ESRF/ID16A/20240924_h5/Chip/Chip_codedht_random2_010nm/'


with  h5py.File(f'{path_out}/data.h5','w') as fid:
    data = fid.create_dataset('/exchange/data',shape=(7200,2048,2048),dtype='uint16')
    data_white0 = fid.create_dataset('/exchange/data_white0',shape=(20,2048,2048),dtype='uint16')
    data_white1 = fid.create_dataset('/exchange/data_white1',shape=(20,2048,2048),dtype='uint16')
    data_dark = fid.create_dataset('/exchange/data_dark',shape=(20,2048,2048),dtype='uint16')
    theta = fid.create_dataset('/exchange/theta',shape=(7200),dtype='float32')
    shifts = fid.create_dataset('/exchange/shifts',shape=(7200,2),dtype='float32')
    shifts[:] = np.loadtxt(f'{path}correct.txt')[:7200]    
    attrs = fid.create_dataset('/exchange/attrs',shape=(7200,3),dtype='float32')
    attrs[:] = np.loadtxt(f'{path}attributes.txt')[:7200]    
    pos_shifts = fid.create_dataset('/exchange/pos_shifts',shape=(7200,2),dtype='float32')
    shifts[:] = np.loadtxt(f'{path}correct.txt')[:7200]    
    dvoxelsize = fid.create_dataset('/exchange/voxelsize',shape=(1,),dtype='float32')
    dvoxelsize[0]=voxelsize
    dsx0 = fid.create_dataset('/exchange/sx0',shape=(1,),dtype='float32')
    dsx0[0] = sx0
    dz1 = fid.create_dataset('/exchange/z1',shape=(1,),dtype='float32')
    dz1[0]=z10
    ddetector_pixelsize = fid.create_dataset('/exchange/detector_pixelsize',shape=(1,),dtype='float32')
    ddetector_pixelsize[0]=detector_pixelsize
    dfocusdetectordistance = fid.create_dataset('/exchange/focusdetectordistance',shape=(1,),dtype='float32')
    dfocusdetectordistance[0]=focusToDetectorDistance



    for id in range(data_white0.shape[0]):
        fname = f'{path}ref{id:04}_0000.edf'
        data_white0[id] = dxchange.read_edf(fname)[0]

    for id in range(data_white1.shape[0]):
        fname = f'{path}ref{id:04}_7200.edf'
        data_white1[id] = dxchange.read_edf(fname)[0]

    for id in range(data_dark.shape[0]):
        fname = f'{path}darkend{id:04}.edf'
        data_dark[id] = dxchange.read_edf(fname)[0]

    for id in range(data.shape[0]):        
        fname = f'{path}Chip_codedht_random2_010nm{id:04}.edf'
        
        ang = find_angle(fname)        
        
        pshifts = find_pos_shifts(fname)
        print(fname)
        print(ang)  
        print(pshifts)  
        data[id] = dxchange.read_edf(fname)[0]
        theta[id] = ang
        pos_shifts[id] = pshifts



['motor_pos', '=', '0', '0.56577', '30', '-2.3428', '0.7093', '-208.025', '2.51', ';']
['motor_pos', '=', '0', '0.56577', '30', '-2.3428', '0.7093', '-208.025', '2.51', ';']
/data/vnikitin/ESRF/ID16A/20240924/Chip/Chip_codedht_random2_010nm/Chip_codedht_random2_010nm0000.edf
0.56577
[-2.3428  0.7093]
['motor_pos', '=', '0.0250169', '0.56577', '30', '-2.3161', '0.6665', '-207.977', '2.56', ';']
['motor_pos', '=', '0.0250169', '0.56577', '30', '-2.3161', '0.6665', '-207.977', '2.56', ';']
/data/vnikitin/ESRF/ID16A/20240924/Chip/Chip_codedht_random2_010nm/Chip_codedht_random2_010nm0001.edf
0.56577
[-2.3161  0.6665]
['motor_pos', '=', '0.0500339', '0.56577', '30', '-2.4244', '0.6417', '-207.952', '2.5425', ';']
['motor_pos', '=', '0.0500339', '0.56577', '30', '-2.4244', '0.6417', '-207.952', '2.5425', ';']
/data/vnikitin/ESRF/ID16A/20240924/Chip/Chip_codedht_random2_010nm/Chip_codedht_random2_010nm0002.edf
0.56577
[-2.4244  0.6417]
['motor_pos', '=', '0.0750508', '0.56577', '30', '-2.3096'